In [1]:
import sys

sys.path.append(".../pipeline_imaging/imaging")  #'/path/to/your/project'

from imaging_maestro2_triton_root import Maestro2_Triton
import imaging_utils
import imaging_classifying_rules
import os

# Folder structure

In [2]:
# Pooled date folders across three sites, after fda processing in the windonw VM
# step1 (pooled)/
# └── Triton/
#     ├── UAB_triton_UAB_triton_20240629-20240706_7001-M-38.fda.zip
#     ├── UAB_triton_UAB_triton_20240629-20240706_7001-M-44.fda.zip
#     ├── UCSD_triton_UCSD_triton_20240629-20240706_1442.fda.zip
#     ├── UW_triton_UCSD_triton_20240629-20240706_1442.fda.zip
#     └── ... (additional zip folders)

# => unzip each folder =>

# folders are unzipped, and ".fda.zip" was replaced to "_fda"
# step2/
# └── Triton/
#     ├── UAB_triton_UAB_triton_20240629-20240706_7001-M-38_fda
#     ├── UAB_triton_UAB_triton_20240629-20240706_7001-M-44_fda
#     ├── UCSD_triton_UCSD_triton_20240629-20240706_1442_fda
#     ├── UW_triton_UCSD_triton_20240629-20240706_1442_fda
#     └── ... (additional unzipped folders)
#         ├──── 2.16.840.1.114517.10.5.1.4.307064520230724155117.1.1.dcm
#         ├──── 2.16.840.1.114517.10.5.1.4.307064520230724155117.2.1.dcm
#         └── ... (additional DICOM files)


#  => triton_triton_instance.organize =>

# DICOM files orangnized by protocols 3 subfolders, two subfolders for unknown protocol and critical info missing (if any)
# protocol names are added to folders and dcm files inside that folder
# step3/
# └── Triton/
#     ├── triton_3d_radial_oct
#     ├── triton_macula_6x6_octa
#     ├── triton_macula12x12_octa
#     ├── unknown_protocol
#     └── critical_info_missing
#         ├──── critical_info_missing_AIREADI_R_UAB_triton_UAB_triton_20240629-20240706_7001-M-39_fda
#         └──── critical_info_missing_AIREADI_L_UAB_triton_UAB_triton_20240629-20240706_7001-M-39_fda
#                 ├──── critical_info_missing_AIREADI_L_UAB_triton_UAB_triton_20240629-20240706_7001-M-39_fda_2.16.840.1.114517.10.5.1.4.307064520230724155117.1.1.dcm
#                 ├──── critical_info_missing_AIREADI_L_UAB_triton_UAB_triton_20240629-20240706_7001-M-39_fda_2.16.840.1.114517.10.5.1.4.307064520230724155117.2.1.dcm
#                 └── ... (additional DICOM files)

#  => triton_triton_instance.onvert =>

# DICOM files are formatted to be NEMA compliant, (only for known 3 protocols,) still organized by protocols. No conversion for unknown protocol and critical_info_missing
# "converted_" is added to the file names, and now there are a list of dicom files in each folder
# step4/
# └── Triton/
#     ├── triton_3d_macula_oct
#     ├── triton_3d_wide_oct
#     └── triton_mac_6x6_octa
#         ├──── converted_triton_mac_6x6_octa_AIREADI_R_UAB_triton_UAB_triton_20240629-20240706_7001-M-39_fda_2.16.840.1.114517.10.5.1.4.307064520230724155149.1.1.dcm
#         ├──── converted_triton_mac_6x6_octa_AIREADI_L_UAB_triton_UAB_triton_20240629-20240706_7001-M-39_fda_2.16.840.1.114517.10.5.1.4.307064520230724155149.2.1.dcm
#         ├──── converted_triton_mac_6x6_octa_AIREADI_R_UAB_triton_UAB_triton_20240629-20240706_7001-M-39_fda_2.16.840.1.114517.10.5.1.4.307064520230724155149.4.1.dcm
#         ├──── converted_triton_mac_6x6_octa_AIREADI_R_UAB_triton_UAB_triton_20240629-20240706_7001-M-39_fda_2.16.840.1.114517.10.5.1.4.307064520230724155149.5.1.dcm
#         └── ... (additional DICOM files)


#  => imaging_utils.format_file => (this process is univeral to all images, not eidon specific)
#  => maestro2_triton_instance.metadata =>

# De-identified again, file renamed, and organized in a final structure for data relase
# step5/
# └── Triton/
#       └──retinal_photography/
#          └─── cfp/
#                └── topcon_triton
#                      ├── 1001
#                      ├── 1002
#                      └─── 4205
#                             ├── 4205_triton_3d_macula_ir_l_2.16.840.1.114517.10.5.1.4.907097520240529143132.2.1.dcm
#                             └── 4205_triton_3d_macula_ir_l_2.16.840.1.114517.10.5.1.4.907097520240529143132.2.1.dcm

#       └──retinal_oct/
#          └─── oct_structural_scan/
#                  └── topcon_triton
#                      ├── 1001
#                      ├── 1002
#                      └─── 4205
#                        ├── 4205_triton_mac_6x6_oct_l_2.16.840.1.114517.10.5.1.4.907097520240529143132.1.1.dcm
#                        └── 4205_triton_mac_6x6_oct_l_2.16.840.1.114517.10.5.1.4.907097520240529143132.1.1.dcm


#       └──retinal_octa/
#          └─── enface/
#                └── topcon_triton
#          └─── flow_cube/
# #              └─ topcon_triton
#          └─── segmentation/
# #         └─ topcon_triton
#                      ├── 1001
#                      ├── 1002
#                      └─── 4205
#                             ├── 4205_triton_mac_6x6_segmentation_l_2.16.840.1.114517.10.5.1.4.907097520240529143132.4.1.dcm
#                             └── 4205_triton_mac_6x6_segmentation_l_2.16.840.1.114517.10.5.1.4.907097520240529143132.4.1.dcm


# step_metadata/
# └── Triton/
#         ├── retinal_photography/
#              ├── filename1.json
#              ├── filename2.json
#             ├── filename3.json
#             └── ... (additional JSON files)
#         ├── retinal_oct/
#              ├── filename1.json
#              ├── filename2.json
#             ├── filename3.json
#              └── ... (additional JSON files)
#         ├── retinal_octa/
#              ├── filename1.json
#              ├── filename2.json
#              ├── filename3.json
#              └── ... (additional JSON files)

# For this, we have to change one .py in the pydicom package  (you can run this multiple times, it will lead to the same outcome)

In [3]:
# import imaging_utils

# # path to the pydicom _dicom_dict.py file in your environment's pydicom package
# file_path = (
#     ".../pipeline_imaging/.env/lib/python3.12/site-packages/pydicom/_dicom_dict.py"
# )

# imaging_utils.update_pydicom_dicom_dictionary(file_path)

In [4]:
from pydicom.datadict import DicomDictionary, keyword_dict
from pydicom.dataset import Dataset


# Define items as (VR, VM, description, is_retired flag, keyword)
#   Leave is_retired flag blank.
new_dict_items = {
    0x0022EEE0: (
        "SQ",
        "1",
        "En Face Volume Descriptor Sequence",
        "",
        "EnFaceVolumeDescriptorSequence",
    ),
    0x0022EEE1: (
        "CS",
        "1",
        "En Face Volume Descriptor Scope",
        "",
        "EnFaceVolumeDescriptorScope",
    ),
    0x0022EEE2: (
        "SQ",
        "1",
        "Referenced Segmentation Sequence",
        "",
        "ReferencedSegmentationSequence",
    ),
    0x0022EEE3: ("FL", "1", "Surface Offset", "", "SurfaceOffset"),
}

# Update the dictionary itself
DicomDictionary.update(new_dict_items)


# Update the reverse mapping from name to tag
new_names_dict = dict([(val[4], tag) for tag, val in new_dict_items.items()])
keyword_dict.update(new_names_dict)




# Triton instance

In [ ]:
maestro2_triton_instance = Maestro2_Triton()

In [6]:
device = "Triton"
main = f"/Volumes"
step1 = f"{main}"
step2 = f"{main}/step2"
step3 = f"{main}/step3"
step4 = f"{main}/step4"
step5 = f"{main}/step5"
step_metadata = f"{main}/metadata"

# Unzip each folder, same name 

In [7]:
zips = imaging_utils.list_zip_files(f"{step1}/{device}")

for zip in zips:
    unzip_file = imaging_utils.unzip_fda_file(zip, step2)

# Organize folders by protocol

In [8]:
folders = imaging_utils.list_subfolders(f"{step2}/{device}")

for folder in folders:
    organize_result = maestro2_triton_instance.organize(folder, f"{step3}/{device}")

# Convert DICOM files to NEMA compliant DICOM files

In [ ]:
protocols = [
    "triton_3d_radial_oct",
    "triton_macula_6x6_octa",
    "triton_macula_12x12_octa",
]

for protocol in protocols:

    output = f"{step4}/{device}/{protocol}"
    if not os.path.exists(output):
        os.makedirs(output)

    folders = imaging_utils.list_subfolders(f"{step3}/{device}/{protocol}")

    for folder in folders:
        print(folder)
        maestro2_triton_instance.convert(folder, output)

# Files are de-identified again, file renamed, and organized in a final structure for data relase + output metadata

In [12]:
for folder in [f"{step4}/{device}"]:
    filelist = imaging_utils.get_filtered_file_names(folder)

    for file in filelist:
        full_file_path = imaging_utils.format_file(file, f"{step5}/{device}")
        maestro2_triton_instance.metadata(full_file_path, f"{step_metadata}/{device}")